# LLM Information Extraction
國立中山大學 112-2 社群媒體分析 課堂作業 

學生：黃奕瑋

使用 Langchian 套件與 HuggingFace 模型來實作：
1. LLM Inference 基本範例
2. Prompt Engineering
3. Information Extraction(Sentiment classification, NER)

## Packet Install

In [ ]:
import os
from getpass import getpass

import pandas as pd
import ast
import json
import re
import jieba

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import BitsAndBytesConfig # huggingface 量化

from langchain_community.llms import HuggingFaceEndpoint

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [2]:
import torch
torch.cuda.is_available()

True

## HuggingFace Token

In [3]:
# 輸入 huggingface 的 token
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

## 載入 LLM 模型

In [4]:
# 設定量化配置，可以用來減少模型記憶體占用
quant_config = BitsAndBytesConfig(load_in_4bit=True)
# 選擇要使用的模型
model_id = "google/gemma-1.1-7b-it"
# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HUGGINGFACEHUB_API_TOKEN)
# 設定 device_map="auto" 讓模型自動選擇 GPU / CPU
device_map="auto" 
# 載入語言模型
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quant_config, token=HUGGINGFACEHUB_API_TOKEN)

# 建立 text-generation pipeline，讓模型可以直接產生文本
# max_new_tokens 生成文本的最大長度
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
# 使用 HuggingFacePipeline 封裝 pipeline 物件，讓它符合 langchain 的接口
llm = HuggingFacePipeline(pipeline=pipe)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\win90\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


### HuggingFaceEndpoint
hugguingfaceEndpoint 是一個 langchain 的 LLM 物件，能夠讓開發者輕鬆地調用 Hugging Face 平台中的機器學習模型。

In [5]:
# 透過 Hugging Face API 部署的 Endpoint 來呼叫模型
# temperature: 模型回答的創意程度，0~1 越大每次回答的多樣性越高
llm = HuggingFaceEndpoint(
    repo_id=model_id, temperature=1.0, model_kwargs={'use_cache':False}, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

c:\Users\win90\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\win90\.cache\huggingface\token
Login successful


### Chain
- LCEL (LangChain Expression Language)
    - 官網介紹：https://python.langchain.com/v0.1/docs/expression_language/why/
    - Langchain 說明：LCEL 可以讓我們更簡單的開發複雜的 chain 應用，可以支援：Streaming, Parallelism, logging。
    - 為了簡單化，LCEL 有兩的主要功能
        1. 統一的協定(Protocol)：所有的 LCEL object 都要遵照 Runnable interface，簡單來說都要包含以下方法，來讓多個 LCEL object 之間可以 chain(鏈結起來):
            1. invoke (支援單一輸入、單一輸出）
            2. batch (支援多個輸入、多個輸出）
            3. stream (支援有部分結果就輸出的模式)
            - 詳細定義：
                1. https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable
                2. https://python.langchain.com/v0.1/docs/expression_language/interface/
        2. 組合語言：讓 chain 的組合更容易
            - 提供不同元件組合的功能，使用 "|" 將不同元件串在一起，例如：prompt | llm
- Chain
    - 可重複執行
    - 可自由組合
    - 有兩種主要的型態
        1. RunnableSequence: 依序執行每個 Runnable
        2. RunnableParallel: 同時執行 Runnable

In [6]:
print(llm.invoke("自然語言處理是什麼？"))



自然語言處理（NLP）是一種用於理解和產生人類語言的計算機技術。它是一種與人類語言學和計算機科學交匯的領域，專注於開發技術，使計算機能夠理解和產生自然語言。

**NLP 的目的：**

* 讓計算機理解人類語言的含義。
* 讓計算機生成人類語言。

**NLP 的應用：**

* 語言翻譯
* 文本摘要
* 情感分析
* 自然語言查詢
* 語音辨識


**NLP 的技術：**

* 語彙分析：分析語言結構。
* 語法分析：分析語言語法。
* 詞彙庫建置：建立語言中的詞彙庫。
* 情感分析：分析語言的情感內容。


**NLP 的未來：**

NLP 領域正在不斷發展，並有望在未來應用更廣泛。隨著技術的進步，我們可以預期 NLP 在以下領域有更大的應用：

* 人工智慧輔助
* 教育和學習
* 健康診斷
* 自動化工作流程


**結論：**

自然語言處理是一種先進的計算機技術，用於理解和產生人類語言。其應用廣泛，並在未來有更大的應用，以促進人類與計算機之間的溝通。


#### RunnableSequence
- 依序執行每個 Runnable
- 使用 "|" 連結

使用 PromptTemplate 建立 Prompt 模板

In [7]:
# 這裡的 {question} 是一個變數，會在執行時被替換成實際的問題
template = """你是一位 {role}，請幫以下句子的評分情緒分數（介於 -1.0~1.0，越高分代表越正向，反之越負向）: {question}"""

# 使用 PromptTemplate 來建立模板，這樣可以讓我們在執行時動態地替換變數
prompt = PromptTemplate.from_template(template)

# 利用 dictionary 來傳遞變數的值，這樣就可以在執行時替換模板中的變數
prompt_output = prompt.invoke({"role" : "社群評論專家", "question": '這家餐廳的料理廚藝了得！'})

In [ ]:
print(prompt_output)
print(llm.invoke(prompt_output))

建立 RunnableSequence：
1. RunnablePassthrough() ： 在 invoke 時將 value 丟進來
2. prompt.invoke(...) 
3. llm.invoke(...)

In [8]:
# 把東西丟進 question，出來的結果再丟進 prompt，結果再丟進 llm
sentiment_value_chain = ( 
    {"role" : RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [ ]:
result = sentiment_value_chain.invoke({"role" : '社群評論專家', "question": '這家餐廳的料理廚藝了得！'})
print(result)

#### RunnableParallel
- 同時執行多個 Runnable
- 使用 "|" 連結

情緒分析 function 設定

In [9]:
# 設定繁體中文詞庫
jieba.set_dictionary("./dict/dict.txt.big")

# 新增stopwords
# jieba.analyse.set_stop_words('./dict/stop_words.txt') #jieba.analyse.extract_tags才會作用
with open("./dict/stop_words.txt", encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]

# 設定斷詞 function
def getToken(row):
    seg_list = jieba.cut(row, cut_all=False)
    seg_list = [
        w for w in seg_list if w not in stopWords and len(w) > 1
    ]  # 篩選掉停用字與字元數大於1的詞彙
    return seg_list

# 讀取情緒字典
liwc_dict = pd.read_csv("./dict/liwc/LIWC_CH.csv")
liwc_dict = liwc_dict.rename(columns={'name': 'word', "class": 'sentiments'})
# call lexicon base function
# 情緒分析
def lexicon_base_sentiment_value(value: dict) -> str:
    tokens = getToken(value['question'])
    df = pd.DataFrame(tokens, columns=['word'])
    liwc_df = pd.merge(df, liwc_dict, how="left")
    sentiment_count = pd.DataFrame(
        liwc_df.groupby(["sentiments"]).size()
    ).reset_index()
    result_dict = sentiment_count.set_index('sentiments')[0].to_dict()
    print(result_dict)
    s_value = (result_dict.get('positive') if result_dict.get('positive') != None else 0) - (result_dict.get('negative') if result_dict.get('negative') != None else 0)
    return s_value


In [11]:
lexicon_sentiment_value_chain = RunnableLambda(lexicon_base_sentiment_value)

# 同時執行 情緒分析 和 RunnableSequence
parallel = {"runnable_1": lexicon_sentiment_value_chain, "runnable_2": sentiment_value_chain}
chain = RunnableLambda(lambda x: x) | parallel
# chain = RunnableParallel(lexicon=lexicon_sentiment_value_chain, LLM=sentiment_value_chain)
result = chain.invoke({"role" : '社群評論專家', "question": '這家餐廳的料理廚藝了得！'})

print(result)

{}
{'runnable_1': 0, 'runnable_2': '。\n\n評分情緒分數：0.8\n\n**評估說明：**這句子的情緒分數為 0.8，代表這句子表示對料理廚藝的肯定，並且帶有愉悅的情緒。'}


### Prompt templete
- 官網：https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/
- Prompt templete 可以預先定義一個要輸入給 LLM 的文字模板
- Langchian prompt templete 通常有兩種型態：
    1. PromptTemplate: string (前面用的)
    2. ChatPromptTemplate: list of chat messages

#### ChatPromptTemplate: list of chat messages
- 內容為 chat message，當中有 "role" 角色的概念，讓 LLM 了解內容是什麼角色說的
- system: 這個role負責的是類似系統設定的文字提示，例如跟 LLM 說他是一個工程師，接下來要請他用他的專業寫 code；或是跟 LLM 說他是一個厲害的數學家，要請他用他的專業解題。
- user: 使用者說的話，例如問 LLM 的問題。

In [13]:
# 輸入一連串的對話模板
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

# 從 print 出文字可以看出，有 {name} 和 {user_input} 兩個變數
print(chat_template.input_schema.schema())

{'title': 'PromptInput', 'type': 'object', 'properties': {'name': {'title': 'Name', 'type': 'string'}, 'user_input': {'title': 'User Input', 'type': 'string'}}}


[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='What is your name?')]

In [16]:
# 執行模板，並傳入變數的值
chat_prompt = chat_template.invoke({"name": "Bob", "user_input":"What is your name?"})

print(chat_prompt)
print(chat_prompt.to_string())

messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')]
System: You are a helpful AI bot. Your name is Bob.
Human: Hello, how are you doing?
AI: I'm doing well, thanks!
Human: What is your name?


# Prompt engineering
- Prompt 的設計有很大程度會影響 LLM 的輸出，甚至影響 LLM 整體能力，以下會介紹幾個常用的 prompt 技巧
    1. Few-shot
    2. Chain of thought(CoT)
    3. Self-consistency

## 一般的 Prompt

In [23]:
normal_template = """<start_of_turn>user
你是一位數學家，將會幫助回答數學問題。

問題：{question}<end_of_turn>
<start_of_turn>model
答案："""

normal_prompt = PromptTemplate.from_template(normal_template)

normal_chain = (
    {"question": RunnablePassthrough()}
    | normal_prompt
    | llm
)

In [ ]:
result = normal_chain.invoke("當我六歲的時候，我妹妹的年齡只有我的一半。 現在我70歲了，我妹妹幾歲了？")
print(result)

## Few shot

In [18]:
fewshot_template = """<start_of_turn>user
你是一位數學家，將會幫助回答數學問題。

以下為一些例子：
```
問題：當我九歲的時候，我弟弟的年齡是我的三分之二。現在我54歲了，我弟弟幾歲了？
答案：51歲

問題：當我八歲的時候，我妹妹的年齡是我的四分之一。現在我48歲了，我妹妹幾歲了？
答案：42歲
```

現在請回答以下問題
問題：{question}<end_of_turn>
<start_of_turn>model
答案："""

fewshot_prompt = PromptTemplate.from_template(fewshot_template)

fewshot_chain = (
    {"question": RunnablePassthrough()}
    | fewshot_prompt
    | llm
)

In [20]:
result = fewshot_chain.invoke("當我六歲的時候，我妹妹的年齡只有我的一半。 現在我70歲了，我妹妹幾歲了？")
print(result)

60歲。

解法：
如果我六歲的時候，我的妹妹的年齡只有我的一半，則我的妹妹比我小3歲。

現在我70歲了，我的妹妹比我小3歲，則我的妹妹現在60歲了。


## Chain of Thought (COT)

In [21]:
CoT_template = """<start_of_turn>user
你是一位數學家，將會幫助回答數學問題。

以下為一些例子：
```
例子1:
問題：當我九歲的時候，我弟弟的年齡是我的三分之二。現在我54歲了，我弟弟幾歲了？
答案：我們一步一步來計算。

當你九歲的時候，你弟弟的年齡是你的三分之二：

你9歲時，你弟弟是 9x(2/3) = 6歲。
現在你54歲了：

你比當時大了45歲（54 - 9 = 45）。
因此，你弟弟現在也比當時大了45歲。
計算你弟弟的年齡：

你弟弟當時6歲，現在他比當時大45歲，所以他現在是6 + 45 = 51歲。
所以，你弟弟現在51歲。


例子2:
問題：當我八歲的時候，我妹妹的年齡是我的四分之一。現在我48歲了，我妹妹幾歲了？
答案：
我們一步一步來計算。

當你八歲的時候，你妹妹的年齡是你的四分之一：

你8歲時，你妹妹是 8*(1/4)=2 歲。
現在你48歲了：

你比當時大了40歲（48 - 8 = 40）。
因此，你妹妹現在也比當時大了40歲。
計算你妹妹的年齡：

你妹妹當時2歲，現在她比當時大40歲，所以她現在是2 + 40 = 42歲。
```

現在請回答以下問題
問題：{question}<end_of_turn>
<start_of_turn>model
答案："""

CoT_prompt = PromptTemplate.from_template(CoT_template)

CoT_chain = (
    {"question": RunnablePassthrough()}
    | CoT_prompt
    | llm
)

In [22]:
result = CoT_chain.invoke("當我六歲的時候，我妹妹的年齡只有我的一半。 現在我70歲了，我妹妹幾歲了？")
print(result)



我們一步一步來計算。

當你六歲的時候，你妹妹的年齡是你的一半：

你6歲時，你妹妹是 6/2 = 3歲。
現在你70歲了：

你比當時大了64歲（70 - 6 = 64）。
因此，你妹妹現在也比當時大了64歲。
計算你妹妹的年齡：

你妹妹當時3歲，現在她比當時大64歲，所以她現在是3 + 64 = 67歲。

所以，你妹妹現在67歲。


由此可以發現，在數學問題上，COT 可以有效提高模型思考能力。

## Self-Consistency

In [24]:
for i in range(1, 4):
  print(f"\n\n========== Run {i} ==========\n\n")
  print(normal_chain.invoke("當我六歲的時候，我妹妹的年齡只有我的一半。 現在我70歲了，我妹妹幾歲了？"))



========== Run 1 ==========


**50歲**

設現在你的妹妹的年齡為x歲，而你的年齡為70歲。根據給定的資訊，我們知道：

當你六歲的時候，你的妹妹的年齡只有你的一半，因此：
x-6 = (70-6)/2

解出x，得到：
x-6 = 32
x = 38

因此，現在你的妹妹年齡為**38歲**。


========== Run 2 ==========


**50歲**

**步驟：**

1. 設我當時的年齡為 x 年。
2. 妹妹的年齡為 x/2 年。
3. 我的現在年齡為 70 年。
4. 那麼妹妹的現在年齡為 (x/2) + (70 - x)。
5. 因為她的年齡始終為我的一半，所以： (x/2) + (70 - x) = (1/2)x。
6. 解出 x 的值，得到 x = 50。

因此，妹妹現在 50 歲。


========== Run 3 ==========


**50歲**

**解法：**

設你的年齡現在為 x 年。

當你六歲時，你的妹妹的年齡為 x/2 年。

現在你的年齡為 70 年，因此：

$$x-6=70$$

$$x=70+6$$

$$x=76$$

你的妹妹的年齡現在為 (76-6)/2 = **50歲**。


## Few shot + Self-consistency

In [25]:
for i in range(1, 4):
  print(f"\n\n========== Run {i} ==========\n\n")
  print(fewshot_chain.invoke("當我六歲的時候，我妹妹的年齡只有我的一半。 現在我70歲了，我妹妹幾歲了？"))



========== Run 1 ==========


65歲

解釋：

設我妹妹的年齡為x歲。

根據問題中的描述，我們有以下等式：

```
x = (1/2) * 70
```

解出x，得到：

```
x = 35
```

因此，現在我的妹妹的年齡為**35歲**。


========== Run 2 ==========


65歲

設下以下變量：

* 我妹妹的年齡為 x 年。

根據給定的資訊，我們知道：

* 當我六歲的時候，x = (1/2) * 6 = 3 年。
* 我現在 70歲了，所以我當時距今 70 - 6 = 64 年。

因此，我的妹妹現在為：

```
x + 64 = 3 + 64 = 67
```

所以，我的妹妹現在 67歲了。


========== Run 3 ==========


65歲

解法：
如果我六歲的時候，我妹妹的年齡只有我的一半，則我妹妹的年齡為6/2=3歲。
從那以後，我妹妹的年齡比我少3歲。
現在我70歲了，我妹妹的年齡為70-3=65歲。


原先Few shot 以及 Self-consistency 都沒有正確答案。 <br>
但兩者結合後，就有機會算出正確解答。

# RAG

In [26]:
loader = PyPDFLoader("./data/中大學1122選課須知.pdf")
pages = loader.load()
pages
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(pages)
texts[:10]

[Document(page_content='國立中山大學  \n112學年度第 2學期  \n選課須知  \n   \n \n \n \n \n \n \n➢ 辦理依據：依本校學則第十條第三款訂定本 選課須知。 \n➢ 中華民國 112年12月12日本校第 178次教務會議通過  \n➢ 選課系統網址： https://selcrs.nsysu.edu.tw/', metadata={'source': './data/中大學1122選課須知.pdf', 'page': 0}),
 Document(page_content='目     錄 \n \n一、選課相關時程 ……………………………………………….1  \n二、選課前注意事項 ……………………………………………. 2 \n三、選課說明 …………………………………………………….3  \n四、各階段辦理方式及注意事項  \n（一）必修課程確認 ………………………………………. 4 \n（二）超修學分申請 ………………………………………. 4 \n（三）校際選課申請 （本校生）…………………………. 4 \n（四）初選第 1階段 ………………………………………. 4 \n（五）初選第 2階段 ………………………………………. 5\n（六）加退選第 1、2階段 …………………………..……. 5 \n（七）選課異常處理 ………………………………………. 6 \n（八）確認選課紀錄 ………………………………………. 7 \n（九）棄選 …………………………………………………. 7 \n五、選課結果篩選次序 …………………………………………. 7 \n六、選課相關規定 ………………………………………………. 8 \n【附錄】本校課號編碼原則 ………………………………… .…11', metadata={'source': './data/中大學1122選課須知.pdf', 'page': 1}),
 Document(page_content='1 \n 一、 選課相關時程：（請依時程辦理，逾時程除重大因素外，依規定不予補辦）  \n階段  開始日期時間  截止日期時間  \n112-2課程查詢  112年 12 / 27 (三) 13:00  \n必修課程確認  1 / 26 (五) 09:00  3 / 1

In [27]:
embedding_function = SentenceTransformerEmbeddings(model_name="lier007/xiaobu-embedding")

vectorstore = Chroma.from_documents(documents=texts, embedding=embedding_function)

retriever = vectorstore.as_retriever()

c:\Users\win90\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
# query it
query = "初選1什麼時候開始選課？"
docs = vectorstore.similarity_search(query)

# print results
print(docs[0].page_content)

1 
 一、 選課相關時程：（請依時程辦理，逾時程除重大因素外，依規定不予補辦）  
階段  開始日期時間  截止日期時間  
112-2課程查詢  112年 12 / 27 (三) 13:00  
必修課程確認  1 / 26 (五) 09:00  3 / 1 (五) 17:00  
系所輔導學生選課  1 / 26 (五) 09:00  3 / 1 (五) 17:00  
校際選課申請 (本校生 ) 
【請配合開課學校校際選課時
程辦理】 1 / 26 (五) 09:00  3 / 1 (五) 17:00  
超修學分申請  
(上網列印 ，紙本簽核 ) 1 / 26 (五) 09:00  3 / 1 (五) 17:00  
(請自行掌握列印及簽核時間 ) 
【初選】 
初選 1 (限學士班選通識課 程) 1 / 26 (五) 09:00   1 / 30 (二) 17:00  
初選 1《結果公布》  1 / 31 (三) 14:00  
初選 2 2 / 1 (四) 09:00 2 / 5 (一) 17:00  
初選 2《結果公布》 2 / 6 (二) 14:00  
【加退選】  
加退選 1 2 / 22 (四) 09:00   2 / 23 (五) 17:00  
加退選 1《結果公布》 2 / 26 (一) 14:00  
加退選 2 2 / 29 (四) 09:00  3 / 1 (五) 17:00  
加退選 2《結果公布》 3 / 4 (一) 14:00  
【異常處理】  
選課異常處理  
(上網列印 ，紙本簽核 ) 3 / 5 (二) 09:00  3 / 11 (一) 17:00  
(請自行掌握列印及簽核時間 ) 
確認選課紀錄  3 / 5 (二) 09:00   3 / 18 (一) 17:00  
【學分費繳交 】 
學分費繳交 (暫訂 ) 3 / 28 (四) 4 / 8 (一) 
【棄選】  
棄選  
(上網列印 ,紙本簽核 ) 5 / 3 (五) 09:00  5 / 10 (五) 17:00  
(請自行掌握列印及簽核時間 ) 
※各階段辦理方式參閱【 四、各階段辦理方式及注意事項】


In [29]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_prompt_templete = """<start_of_turn>user
"角色": "選課機器人",
"服務內容":
你是一個樂於幫助學生的機器人，關於選課或課程的問題你都可以依照以下文件中的規定來回答學生。\n
現在給予以下相關文件, 你將會基於以下 [文件][文件] 中的相關文件與 [回答規則][回答規則] 中的回答規則來回答學生的問題。\n
\n
[文件]
{context}
[文件]
\n
[回答規則]
只依據 [文件][文件] 中提供的資訊來回答，並且遵照下列的幾點規則：
- 不撒謊或幻想 [文件][文件] 中未明確提供的答案
- 如果不確定答案或答案未明確包含在 [文件][文件] 中，請回答：“我很抱歉，我不知道如何提供幫助。”
- 回答簡短、相關且簡潔的答案
- 回答字數請少於 150 個字
- 請在 [結果]: 之後回答你的答案
[回答規則]

<</SYS>>
現在，基於上述的文件與規則，回答以下 () 中的問題\n[學生]:({question})\n<end_of_turn>
<start_of_turn>model[結果]:"""

rag_prompt = PromptTemplate.from_template(rag_prompt_templete)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)
rag_chain.get_graph().print_ascii()

              +---------------------------------+           
              | Parallel<context,question>Input |           
              +---------------------------------+           
                    ****                ****                
                 ***                        ***             
               **                              ***          
+----------------------+                          **        
| VectorStoreRetriever |                           *        
+----------------------+                           *        
            *                                      *        
            *                                      *        
            *                                      *        
+---------------------+                     +-------------+ 
| Lambda(format_docs) |                     | Passthrough | 
+---------------------+                     +-------------+ 
                    ****                ****                
                        

In [30]:
rag_chain.invoke("初選2什麼時候開始選課？")

' 初選 2 的選課開始時間為 2/1 (四) 09:00。'